# 0. Imports and Data

In [1]:
import os
import ast
import re
import json
from IPython.display import display, HTML
import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt
import torch
import pickle
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from transformers import pipeline, AutoTokenizer, AutoModelForQuestionAnswering, BigBirdTokenizer


In [2]:
X_train = pd.read_csv("x_train_Meacfjr.csv", delimiter=",", quotechar='"')
y_train = pd.read_csv("y_train_SwJNMSu.csv", delimiter=",", quotechar='"')

# Convert 'job_ids' and 'actions' from strings to actual lists
X_train["job_ids"] = X_train["job_ids"].apply(ast.literal_eval)
X_train["actions"] = X_train["actions"].apply(ast.literal_eval)

with open("content/job_listings/job_listings.json", "r") as file:
    jobs = json.load(file)

In [3]:
print(jobs["22082"])#, "\n", jobs["127"])

TITLE
CTO (H/F)

SUMMARY
CTO (H/F)

SECTION:
TITLE: Job description
DESCRIPTION: Rattaché(e) au Secrétaire Général, vous pilotez la stratégie informatique de la société et le budget associé. Vous dirigez l'ensemble du SI, coordonnez le suivi des prestataires informatiques et animez les équipes.
Vous incarnez toute la stratégie IT et, à ce titre, occupez un poste polyvalent et responsabilisant.
? Stratégie :
Vous définissez et mettez en œuvre la stratégie IT avec un axe particulier sur des projets applicatifs et BI.
? Mise en œuvre de la stratégie :
- Définition des politiques IT : mise en place et suivi des politiques d'administration, de monitoring, de maintenance et de sauvegarde du SI.
- Infrastructure : Animation de l'équipe de 2 personnes / Gestion des prestataires / Gestion du parc informatique et téléphonique / Supervision des budgets et des contrats / Travaux de mise à niveau de version logicielle...
- Gestion de projets applicatifs : vous participez activem

# 1. Data Preprocessing

## 1.1 Split Job Description

In [4]:
def extract_info(job_id, text):
    
    fulltext = text
    main = text.split("SECTION")[0]
    
    # Extract title
    title_match = re.search(r"TITLE\n(.+)", text)
    title = title_match.group(1).strip() if title_match else None
    
    # Extract summary
    summary_match = re.search(r"SUMMARY\n([\s\S]+?)(?:\nSECTION:|$)", text)
    summary = summary_match.group(1).strip() if summary_match else None
    
    # Extract descriptions
    descriptions = re.findall(r"SECTION:\nDESCRIPTION: ([\s\S]+?)(?:\nSECTION:|$)", text)
    
    # Extract skills only from the SKILLS section
    skills_match = re.findall(r"SKILLS\n(?:- (\{.*?\})\n?)+", text)
    skills = [
        ast.literal_eval(skill)["name"]
        for match in skills_match for skill in re.findall(r"- (\{.*?\})", match)
        if "'name':" in skill
    ]
    
    # Extract tasks only from the TASKS section
    tasks_match = re.findall(r"TASKS\n(?:- (\{.*?\})\n?)+", text)
    tasks = [
        ast.literal_eval(task)["name"]
        for match in tasks_match for task in re.findall(r"- (\{.*?\})", match)
        if "'name':" in task
    ]
    
    # Extract languages, certifications (currently empty sections)
    languages = []
    certifications = []
    
    # Extracted data
    row = {
        "job_id": job_id,
        "title": title,
        "summary": summary,
        "descriptions": "\n\n".join(descriptions),
        "languages": "\n".join(languages),
        "certifications": "\n".join(certifications),
        "skills": "\n".join(skills),  
        "tasks": "\n".join(tasks),
        "fulltext": fulltext,
        "main": main,
        "embedding": []
    }
    return row

In [5]:
rows = []
for job_id in jobs.keys():
    row = extract_info(job_id, jobs[job_id])
    rows.append(row)
df_jobs = pd.DataFrame(rows) 

print(df_jobs.shape)
print(df_jobs.info())

(21917, 11)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21917 entries, 0 to 21916
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   job_id          21917 non-null  object
 1   title           21913 non-null  object
 2   summary         21545 non-null  object
 3   descriptions    21917 non-null  object
 4   languages       21917 non-null  object
 5   certifications  21917 non-null  object
 6   skills          21917 non-null  object
 7   tasks           21917 non-null  object
 8   fulltext        21917 non-null  object
 9   main            21917 non-null  object
 10  embedding       21917 non-null  object
dtypes: object(11)
memory usage: 1.8+ MB
None


In [6]:
df_empty = df_jobs[df_jobs.title.isna()]
display(HTML(df_empty.to_html()))

In [7]:
df_jobs['summary'] = df_jobs['summary'].fillna("nan")
df_jobs['title'] = df_jobs['title'].fillna("nan")

In [8]:
print(df_jobs['main'].apply(len).value_counts().sort_index().head(5))

main
0     3
15    5
16    3
18    1
20    2
Name: count, dtype: int64


## 1.2 Create Features - title, location, seniority, summary, company, industry, contract type, linguas exigidas

In [9]:
# Define functions to call LLM

In [10]:
# Define prompt

In [11]:
# Run testings

In [12]:
# Check if the results are jsons and that the answers are ok

In [13]:
# R

In [14]:
display(HTML(df_jobs.head(10).to_html()))

## 1.3 Embedding of the Job Ads

In [15]:
# Check if CUDA is available and use GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load pre-trained sentence transformer model and move it to the GPU if available
model = SentenceTransformer('all-MiniLM-L6-v2').to(device)

# Define features and weights
features = ['title', 'descriptions']  # Example with multiple features
feature_weights = {'title': 1.0, 'descriptions': 0.8}  # Weights will be applied later
action_weights = {'apply': 2, 'view': 1}

In [16]:
def compute_job_embeddings(df, features):
    job_embeddings = {feature: [] for feature in features}  # Store embeddings per feature

    # Compute all embeddings
    for idx, row in df.iterrows():
        if idx % 100 == 0:
            print(idx)
        for feature in features:
            feature_vector = model.encode(str(row[feature]), convert_to_tensor=True, show_progress_bar=False).to(device)
            job_embeddings[feature].append(feature_vector)

    # Convert to PyTorch tensors each feature
    for feature in features:
        job_embeddings[feature] = torch.stack(job_embeddings[feature])

    # Zero-mean normalization for each feature
    for feature in features:
        mean = torch.mean(job_embeddings[feature], dim=0)
        job_embeddings[feature] = job_embeddings[feature] - mean

    return job_embeddings

In [22]:
df = df_jobs.sample(100)

# Load or Compute Embeddings
embedding_path = "data/job_embeddings.pth"
# if os.path.exists(embedding_path):
#     print("Loading embeddings from {}".format(embedding_path))
#     job_embeddings = torch.load(embedding_path, map_location=device)
# else:
print("Computing embeddings...")
job_embeddings = compute_job_embeddings(df, features)
torch.save(job_embeddings, embedding_path)

# Add embeddings to DataFrame
for feature in features:
    df[f'embedding_{feature}'] = list(job_embeddings[feature])

# Create a mapping from job_id to its embeddings (per feature)
job_embedding_dict = {row['job_id']: {feature: row[f'embedding_{feature}'] for feature in features}
                      for _, row in df.iterrows()}

Computing embeddings...
21264
15291
18369
13298
15531
9236
15896
14330
13236
11016
19984
2043
12266
7721
4642
14380
15524
784
20214
21614
18909
13799
8318
7813
18591
16845
16253
15077
5453
5123
9033
12607
11557
5481
17865
15176
727
18124
2470
20010
14845
13949
21638
9742
20746
11675
3726
20186
17866
5025
4752
19074
15827
8286
13091
14734
14935
12071
12221
14422
21258
5439
1902
12777
17288
19727
15828
15623
10731
14772
18966
16003
20529
7071
8921
17780
18170
7016
5622
10135
9765
14696
399
13433
8076
8027
16186
2838
2725
4948
9494
13566
538
9247
20235
837
761
8195
13611
13423


In [ ]:
# Function to generate recommendations for sessions dataframe using GPU
def recommend_jobs_for_sessions(x, df_jobs, job_embedding_dict, feature_weights, action_weights, top_n=10):
    recommendations = {}

    for idx, row in x.iterrows():
        if idx % 100 == 0:
            print(f"Processing session {idx}")
        
        session_id = row['session_id']
        job_ids = [int(job_id) for job_id in row['job_ids']]
        actions = row['actions']

        # Compute user profile by aggregating weighted embeddings of interacted job ads
        session_embeddings = []
        
        for job_id in job_ids:
            job_features = job_embedding_dict[job_id]  # Dict {feature: tensor}
            weighted_embedding = torch.cat([
                job_features[feature] * feature_weights[feature] for feature in feature_weights
            ], dim=0)  # Concatenate weighted feature embeddings
            
            session_embeddings.append(weighted_embedding)

        session_embeddings = torch.stack(session_embeddings).to(device)  # Convert to tensor on GPU
        weights = torch.tensor([action_weights[action] for action in actions], dtype=torch.float32).to(device).view(-1, 1)

        # Compute user profile vector (weighted average of past interactions)
        user_profile_vector = torch.sum(session_embeddings * weights, dim=0) / torch.sum(weights)

        # Compute similarity with all job embeddings
        job_embeddings_tensor = []
        for _, job_row in df_jobs.iterrows():
            job_embedding = torch.cat([
                job_row[f'embedding_{feature}'] * feature_weights[feature] for feature in feature_weights
            ], dim=0)  # Concatenate weighted feature embeddings
            
            job_embeddings_tensor.append(job_embedding)

        embeddings_matrix = torch.stack(job_embeddings_tensor).to(device)
        similarities = torch.nn.functional.cosine_similarity(user_profile_vector.unsqueeze(0), embeddings_matrix)

        # Exclude already interacted job IDs
        recommended_jobs = df_jobs.copy()
        recommended_jobs['similarity'] = similarities.cpu().numpy()
        recommended_jobs = recommended_jobs[~recommended_jobs['job_id'].isin(job_ids)]

        # Store top N recommendations
        recommendations[session_id] = recommended_jobs[['job_id', 'similarity']].sort_values(
            by='similarity', ascending=False).head(top_n)['job_id'].values

    return recommendations

In [ ]:
if os.path.exists("content/y_train_predict.pkl"):
    with open("recommendations.pkl", "rb") as f:
        y_predict_dict = pickle.load(f)
else:
    y_predict_dict = recommend_jobs_for_sessions(X_train, df_jobs, job_embedding_dict, action_weights)
    with open("content/y_train_predict.pkl", "wb") as f:
        pickle.dump(y_predict_dict, f)

In [ ]:
def loss_train(y, y_predict_dict):
    reciprocal_ranks = []
    for _, row in y.iterrows():
        try:
            recomendations = y_predict_dict[row['session_id']]
            recomendations = [int(r) for r in recomendations]
            target_value = int(row['job_id'])
            if target_value in recomendations:
                rank = recomendations.index(target_value) + 1
                rr = 1.0 / rank
            else:
                rr = 0.0
        except Exception as e:
            print(e)
            rr = 0.0
        reciprocal_ranks.append(rr)
        
    df = pd.DataFrame(reciprocal_ranks, columns=['r'])
    print(df["r"].value_counts().sort_index())
    
    mrr = np.mean(reciprocal_ranks)
    print(f"Mean Reciprocal Rank (MRR): {mrr:.4f}")
    
    return mrr

In [ ]:
# print(session_recommendations[15869])
loss_train(y_train, y_predict_dict)